### Ejemplo de extracción de datos estructurados: Dark Sky

In [1]:
# cargamos las librerias
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import json
import urllib.request
import pandas as pd
import time
import json
import numpy as np

#### Función para hacer la descarga

In [2]:
def api_dark_ski(coor, begin_date, end_date):
    #----------------------------------------------------------
    ## Función para descargar el clima historico de un conjunto
    ## de lugares con coordenadas geograficas.
    #
    #
    # Parametros
    # - coor:       es un datafram con columnas: state, lat, lon.
    # - begin_date: es la fecha de inicio para descargar.
    # - end_date:   es la fecha final para descargar.
    
    # creamos el vector de las fechas.
    fechas = pd.date_range(start=pd.to_datetime(begin_date), 
                           end=pd.to_datetime(end_date),
                           freq="D")
    fechas_unix= (fechas-pd.to_datetime("1970-01-01")).total_seconds() # formato especifico de la API.
    fechas_unix = fechas_unix.astype("int")
    
    fr = [] # inicializamos los dataframe
    clima = pd.DataFrame()
    
    # iteramos por cada estado
    for num, i in enumerate(coor["state"]):
        print(i) # imprimimos el estado descargando.
        fr.append(i)
        for date in fechas_unix:   
            print(date)
            x = coor[coor.state==i]["lat"][num]
            y = coor[coor.state==i]["log"][num]
            weather = "https://api.darksky.net/forecast/93a3ab8136ea441147c4702bc5bdabc3/{},{},{}?exclude=[currently,minutely,hourly,alerts,flags]".format(x, y , date)
            page = urllib.request.urlopen(weather).read()
            jsonResponse = json.loads(page.decode('utf-8'))
            if "daily" in jsonResponse.keys():
                daily = jsonResponse["daily"]["data"][0]
                temp = pd.DataFrame.from_dict(jsonResponse["daily"]["data"])
                temp["franchise"] = i
                temp["date"] = date
                temp["time"] = temp.time.apply(lambda x: time.strftime("%D %H:%M", time.localtime(int(x))))
                clima = pd.concat([clima,temp])

    print("------------------------------------------------------------")
    print("Descarga finalizada")
    return clima

## Parametros para la descarga

In [4]:
# leemos los datos de las coordenados.
coor_lugares = pd.read_csv("../data/coordenadas.csv")
coor_lugares = coor_lugares.head(3) # para no hacer muchas descargas cortamos los datos


# definimos las fechas de descarga
begin_date = "2020-04-28"
end_date = "2020-04-30"

### Hacemos la descarga

In [5]:
clima = api_dark_ski(coor_lugares, begin_date, end_date)

Culiacán
1588032000
1588118400
1588204800
Mexicali
1588032000
1588118400
1588204800
Zapopan
1588032000
1588118400
1588204800
------------------------------------------------------------
Descarga finalizada


In [6]:
clima

,time,summary,icon,sunriseTime,sunsetTime,moonPhase,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,...,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,franchise,date
0,04/27/20 01:00,Clear throughout the day.,clear-day,1587991080,1588037940,0.16,0.0005,0.0022,1587970740,0.01,...,63.58,1587981720,99.45,1588020060,64.07,1587981720,98.95,1588020060,Culiacán,1588032000
0,04/28/20 01:00,Clear throughout the day.,clear-day,1588077420,1588124340,0.19,0.0009,0.0023,1588101300,0.01,...,62.64,1588068060,97.20,1588103580,63.13,1588068060,96.70,1588103580,Culiacán,1588118400
0,04/29/20 01:00,Clear throughout the day.,clear-day,1588163760,1588210800,0.22,0.0009,0.0023,1588150980,0.04,...,64.16,1588154580,97.30,1588190760,64.65,1588154580,96.80,1588190760,Culiacán,1588204800
0,04/27/20 02:00,Clear throughout the day.,clear-day,1587992340,1588040520,0.16,0.0000,0.0000,1587970800,0.00,...,70.91,1587984720,102.65,1588030500,71.40,1587984720,102.15,1588030500,Mexicali,1588032000
0,04/28/20 02:00,Clear throughout the day.,clear-day,1588078680,1588126980,0.19,0.0009,0.0021,1588101240,0.02,...,69.39,1588071780,103.04,1588117980,69.88,1588071780,102.54,1588117980,Mexicali,1588118400
0,04/29/20 02:00,Clear throughout the day.,clear-day,1588165020,1588213380,0.23,0.0008,0.0022,1588168680,0.02,...,70.12,1588158180,104.87,1588199040,70.61,1588158180,104.37,1588199040,Mexicali,1588204800
0,04/27/20 00:00,Clear throughout the day.,clear-day,1587990420,1588036680,0.16,0.0001,0.0003,1588010580,0.04,...,61.23,1587988020,93.99,1588026960,61.72,1587988020,93.49,1588026960,Zapopan,1588032000
0,04/28/20 00:00,Clear throughout the day.,clear-day,1588076760,1588123080,0.19,0.0001,0.0003,1588054980,0.01,...,58.65,1588073460,92.12,1588104480,59.14,1588073460,91.62,1588104480,Zapopan,1588118400
0,04/29/20 00:00,Clear throughout the day.,clear-day,1588163100,1588209480,0.22,0.0000,0.0000,1588171860,0.00,...,57.74,1588160340,95.37,1588190340,58.23,1588160340,94.87,1588190340,Zapopan,1588204800


In [7]:
clima.to_csv("../data/clima_dark_sky.csv")

#### Coordenadas aleatorias

In [34]:
#coor_res_lugares = {"Culiacán":[24.791140, -107.393059] , "Mexicali": [32.636004, -115.472609],
# "Zapopan": [20.675532, -103.350425],
#"Cd. Juárez": [25.491446, -103.593103], "Guadiana": [21.787660, -101.004431],
#"Monterrey": [25.679673, -100.316839], "Saltillo": [25.434403, -100.995242],
#"Tepic": [21.494274, -104.890123], "Pachuca": [20.082222, -98.769787],
#"Cuautla": [18.811864, -98.955281], "Nogales" : [31.295047, -110.944563],
# "Colima": [19.243124, -103.724317], "Tijuana": [32.504789, -117.008928],
#"Los Mochis": [25.785806, -108.996535], "Pacabtun": [20.962789, -89.596569],
#"Cancún": [21.178003, -86.833443],
#"Altamira" : [22.2345, -97.56],
# "Cedis Norte" : [19.4939194, -99.1107483],
# "Cedis Sur" : [19.2546501, -99.1035614],
# "Celaya" : [20.5222, -100.812], "Chalco" : [19.2614, -98.8919],
# "Ciudad Victoria" : [23.44,-99.24], "Coacalco" : [19.6418, -99.0954],
# "Coecillo" : [20.6356, -101.4061], "Costa" : [16.8638,-99.8816],
#"Cuatitlán": [19.6454, -99.2112], "Cuernavaca": [18.90, -99.33],
#"Cuernavaca Monarca": [18.5564, -99.605], "Irapuato": [20.6628, -101.347],
#"León": [21.1236, -101.68], "Minatitlán": [17.97, -94.50],
#"Mixcoac" : [19.376094, -99.187651],
#"Montaña" : [17.545495, -98.574710],
#"Morelia" : [19.7006, -101.186],
#"Oaxaca" : [17.0669, -96.7203],
#"Orizaba" :[18.893343, -96.939785],
#"Orizaba Sureste" : [18.85, -97.1] ,
#"Pacifico": [19.4341106, -99.2002411],
#"Poza Rica": [20.533, -97.45],
#"Puebla": [19.0433400,-98.2019300],
#"Querétaro" : [20.588, -100.388], "Reyes" : [19.363, -98.977],
#"San Juan de Río": [20.2014, -100.0792], "Tapachula": [14.9, -92.2667],
#"Tehuantepec": [16.3203, -95.2412] , "Tenango": [20.353, -98.204],
#"Texcoco": [19.50,-98.82,], "Tlalnepantla" : [19.539,  -99.1933] ,
#"Tlanepantla Poniente": [19.539,  -99.1933], "Tlalpan":[19.15,-99.15],
#"Tlaxcala": [19.33,-98.15], "Tuxtla Gutierrez":[16.75, -93.1167 ],
#"Uruapan":[19.4172, -102.05], "Veracruz":[19.5426,  -96.9137], 
#"Villahermosa":[17.99,-92.95],"Xalapa":[19.5426,  -96.9137],
# "Zamora":[19.99,-102.23],"Zaragoza":[19.412344, -99.0824]
#}
#
#
#coor_lugares = pd.DataFrame.from_dict(coor_res_lugares)
#
#coor_lat = pd.DataFrame(coor_lugares.loc[0]).reset_index().rename(columns={"index":"state", 0:"lat"})
#coor_lon = pd.DataFrame(coor_lugares.loc[1]).reset_index().rename(columns={"index":"state", 1:"log"})
#
#pd.merge(coor_lat, coor_lon).to_csv("../../Documentos/Proyectos/#Extraccion_de_datos_estructurados-Interfaz_de_Programacion_de_Aplicaciones_-APIs-/data/coordenadas.csv", index=False)